In [14]:
import requests
import random
import json

In [15]:
baseUrl = "https://demo-education-registry.xiv.in/registry/api/v1/"
keycloakUrl = "https://demo-education-registry.xiv.in"

# Institute invite

In [ ]:
instituteUserId = "admin"+str(random.randint(0,999999))+"@mailinator.com"
instituteRequestBody = {
    "instituteName": "te",
    "email": instituteUserId,
    "contactNumber": "!23"
}

In [ ]:
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Institute/invite", headers=headers, json=instituteRequestBody)

print(response.text)
assert response.status_code == 200

# get institute token

In [ ]:
def getToken(userId):
    headers = {
        'content-type': 'application/x-www-form-urlencoded',
    }

    data = {
      'client_id': 'registry-frontend',
      'username': userId,
      'password': 'abcd@123',
      'grant_type': 'password'
    }

    response = requests.post('%s/auth/realms/sunbird-rc/protocol/openid-connect/token'%(keycloakUrl), headers=headers, data=data)
    print(response.json())
#     print(response.status_code)
    token = response.json()["access_token"]
    return token


In [ ]:
token = getToken(instituteUserId)
print(token)

# get institute details

In [ ]:

headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%token,
}

response = requests.request("GET", baseUrl + "Institute", headers=headers)

print(response.text)
intituteOsid = response.json()[0]["osid"]
print(intituteOsid)
assert response.status_code == 200


# Affiliate Institute

In [ ]:
# add affiliation details
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%token,
}

data = """{
"affiliation": [
  {
    "medium": "English",
    "board": "cbse",
    "affiliationNumber":  "123",
    "grantYear": "2000",
    "expiryYear":  "3001",
    "classes": ["Class XII"]
  }
]
}"""
# print(data)
response = requests.put('%sInstitute/%s'%(baseUrl, intituteOsid), headers=headers, data=data)
print(response)
print(response.json())
assert response.status_code == 200

In [ ]:
# get institute details
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%token,
}

response = requests.request("GET", baseUrl + "Institute", headers=headers)

print(response.text)
intituteAffiliateOsid = response.json()[0]["affiliation"][0]["osid"]
print(intituteAffiliateOsid)
assert response.status_code == 200

## For multi level attestation flow check this

In [ ]:
# raise attestation request
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%token,
}

params = (
    ('send', 'true'),
)

data = {
    "entityName":"Institute",
    "entityId":"%s"%intituteOsid,
    "name":"instituteAffiliation",
    "propertiesOSID": {
        "affiliation": [intituteAffiliateOsid]
    }
}
response = requests.post(baseUrl+'send', headers=headers, params=params, json=data)
print(response.status_code)
response.json()
assert response.status_code == 200
print(response.json())
assert response.json()["params"]["status"] == "SUCCESSFUL"

# cbse board attest claims

In [ ]:
# get claims for cbse board
# create user manually and attach board-cbse role
boardUserId = "cbse-admin"
boardToken = getToken(boardUserId)
# print(boardToken)
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%boardToken,
}

response = requests.request("GET", baseUrl + "board-cbse/claims", headers=headers)
assert response.status_code == 200
print(response.json())

In [ ]:
# Attesting the claim as GRANT
data = {
    "action":"GRANT_CLAIM",
}
# print(response.json())
attestingClaimId = None
for claim in response.json():
#     print(claim)
    print(claim["status"])
    if claim["status"] == "OPEN":
        attestingClaimId = claim["id"]
        print(claim["id"])
        attestResp = requests.request("POST", "%sboard-cbse/claims/%s/attest" %(baseUrl, claim["id"]), headers=headers, json=data)
        assert response.status_code == 200
        print(attestResp)
        print(attestResp.json())
        break
assert attestingClaimId != None

# intituteAffiliateOsid = response.json()[0]["affiliation"][0]["osid"]
# print(intituteAffiliateOsid)

In [ ]:
import time
time.sleep(5)
# verify attestation status
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%token,
}

response = requests.request("GET", baseUrl + "Institute", headers=headers)

print(response.text)
if "instituteAffiliation" in response.json()[0]:
    for affiliation in response.json()[0]["instituteAffiliation"]:
            if affiliation["_osClaimId"] == attestingClaimId:
                assert affiliation["_osState"] == "PUBLISHED"

In [ ]:
# get institute details
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%token,
}

response = requests.request("GET", baseUrl + "Institute", headers=headers)

print(response.text)
for affiliation in response.json()[0]["instituteAffiliation"]:
        if affiliation["_osClaimId"] == attestingClaimId:
            assert affiliation["_osState"] == "PUBLISHED"

# Teacher attestations


In [ ]:
# create new institutions
institute1 = "ABC"+str(random.randint(0,999999))+"@mailinator.com"
institute2 = "XYZ"+str(random.randint(0,999999))+"@mailinator.com"
instituteRequestBody = {
    "instituteName": "ABC",
    "email": institute1,
    "contactNumber": "ABC1"
}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Institute/invite", headers=headers, json=instituteRequestBody)

print(response.text)
assert response.status_code == 200
institute1Osid = response.json()["result"]["Institute"]["osid"]
instituteRequestBody = {
    "instituteName": "XYZ",
    "email": institute2,
    "contactNumber": "XYZ1"
}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Institute/invite", headers=headers, json=instituteRequestBody)

print(response.text)
assert response.status_code == 200
institute2Osid = response.json()["result"]["Institute"]["osid"]

In [ ]:
# create teacher
teacherUserId = "t7"+str(random.randint(0,999999))
teacherRequestBody = {
  "identityDetails": {
    "fullName": "teacher1",
    "gender": "Male",
    "dob": "1995-10-28",
    "identityHolder": {
      "AADHAAR": "123412323"
    }
  },
  "contactDetails": {
    "email": "%s@g.com"%teacherUserId,
    "mobile": teacherUserId
  },
  "academicQualifications": [
    {
      "instituteOSID": institute1Osid,
      "instituteName": "ABC"
    }
  ],
  "experience": [
    {
      "instituteOSID": institute2Osid,
      "instituteName": "XYZ"
    }
  ]
}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Teacher/invite", headers=headers, json=teacherRequestBody)

print(response.text)
assert response.status_code == 200
teacherOsid = response.json()["result"]["Teacher"]["osid"]

In [ ]:
teacherToken = getToken(teacherUserId)
print(teacherToken)

In [ ]:
# get teacher details
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%teacherToken,
}

response = requests.request("GET", baseUrl + "Teacher", headers=headers)

print(response.text)
assert response.status_code == 200
teacherEntity = response.json()[0]
print(teacherEntity["experience"][0]["osid"])
experienceOsid = teacherEntity["experience"][0]["osid"]
print(teacherEntity["academicQualifications"][0]["osid"])
qualificationOsid = teacherEntity["academicQualifications"][0]["osid"]

# raise teacher's education attestation

In [ ]:
# raise attestation request
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%teacherToken,
}

data = {
    "entityName":"Teacher",
    "entityId":"%s"%teacherOsid,
    "name":"teacherExperience",
    "propertiesOSID": {
        "experience": [experienceOsid]
    }
}
response = requests.post(baseUrl+'send', headers=headers, json=data)
print(response.status_code)
response.json()
assert response.status_code == 200
print(response.json())
assert response.json()["params"]["status"] == "SUCCESSFUL"

In [ ]:
# get institute token
institute1Token = getToken(institute1)
print(institute1Token)
institute2Token = getToken(institute2)
print(institute2Token)

In [ ]:
# get institute claims and grant the claim
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%institute2Token,
}

response = requests.request("GET", baseUrl + "Institute/claims", headers=headers)
assert response.status_code == 200
print(response)

# Attesting the claim as GRANT
data = {
    "action":"GRANT_CLAIM",
}
print(response.json())
attestingClaimId = None
for claim in response.json():
#     print(claim)
    print(claim["status"])
    if claim["status"] == "OPEN":
        attestingClaimId = claim["id"]
        print(claim["id"])
        attestResp = requests.request("POST", "%sInstitute/claims/%s/attest" %(baseUrl, claim["id"]), headers=headers, json=data)
        assert response.status_code == 200
        print(attestResp)
        print(attestResp.json())
        break
assert attestingClaimId != None


In [ ]:
import time
time.sleep(5)
# verify attestation status
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%teacherToken,
}

response = requests.request("GET", baseUrl + "Teacher", headers=headers)

# print(response.text)
for affiliation in response.json()[0]["teacherExperience"]:
        if affiliation["_osClaimId"] == attestingClaimId:
            assert affiliation["_osState"] == "PUBLISHED"


In [ ]:
# raise qualification attestation request
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%teacherToken,
}

data = {
    "entityName":"Teacher",
    "entityId":"%s"%teacherOsid,
    "name":"teacherAcademicsQualification",
    "propertiesOSID": {
        "academicQualifications": [qualificationOsid]
    }
}
response = requests.post(baseUrl+'send', headers=headers, json=data)
print(response.status_code)
response.json()
assert response.status_code == 200
print(response.json())
assert response.json()["params"]["status"] == "SUCCESSFUL"



In [ ]:

# get institute token
institute1Token = getToken(institute1)
print(institute1Token)

# get institute claims and grant the claim
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%institute1Token,
}

response = requests.request("GET", baseUrl + "Institute/claims", headers=headers)
assert response.status_code == 200
print(response)

# Attesting the claim as GRANT
data = {
    "action":"GRANT_CLAIM",
}
print(response.json())
attestingClaimId = None
for claim in response.json():
#     print(claim)
    print(claim["status"])
    if claim["status"] == "OPEN":
        attestingClaimId = claim["id"]
        print(claim["id"])
        attestResp = requests.request("POST", "%sInstitute/claims/%s/attest" %(baseUrl, claim["id"]), headers=headers, json=data)
        assert response.status_code == 200
        print(attestResp)
        print(attestResp.json())
        break
assert attestingClaimId != None

In [ ]:

import time
time.sleep(3)
# verify attestation status
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%teacherToken,
}

response = requests.request("GET", baseUrl + "Teacher", headers=headers)


for affiliation in response.json()[0]["teacherAcademicsQualification"]:
    print(affiliation)
    if affiliation["_osClaimId"] == attestingClaimId:
        assert affiliation["_osState"] == "PUBLISHED"

# Student Attestations

In [ ]:
# create teacher
studentUserId = "s"+str(random.randint(0,999999))
studentRequestBody = {
  "identityDetails": {
    "fullName": "student1",
    "gender": "Male",
    "dob": "1995-10-28",
    "identityHolder": {
      "AADHAAR": "123412323"
    }
  },
  "contactDetails": {
    "email": "%s@g.com"%studentUserId,
    "mobile": studentUserId
  },
  "educationDetails": [
    {
      "instituteOSID": institute2Osid,
      "instituteName": "XYZ"
    }
  ]
}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Student/invite", headers=headers, json=studentRequestBody)

print(response.text)
assert response.status_code == 200
studentOsid = response.json()["result"]["Student"]["osid"]

In [ ]:
studentToken = getToken(studentUserId)
print(studentToken)

In [ ]:
# get student details
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%studentToken,
}

response = requests.request("GET", baseUrl + "Student", headers=headers)

print(response.text)
assert response.status_code == 200
print(response.json()[0]["educationDetails"][0]["osid"])
educationDetailsOsid = response.json()[0]["educationDetails"][0]["osid"]

In [ ]:
# raise qualification attestation request
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%studentToken,
}

data = {
    "entityName":"Student",
    "entityId":"%s"%studentOsid,
    "name":"studentInstituteAttest",
    "propertiesOSID": {
        "educationDetails": [educationDetailsOsid]
    }
}
response = requests.post(baseUrl+'send', headers=headers, json=data)
print(response.status_code)
response.json()
assert response.status_code == 200
print(response.json())
assert response.json()["params"]["status"] == "SUCCESSFUL"


In [ ]:
# get teacher token
teacherToken = getToken(teacherUserId)
print(teacherToken)

# get institute claims and grant the claim
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%teacherToken,
}

response = requests.request("GET", baseUrl + "Teacher/claims", headers=headers)
assert response.status_code == 200
print(response)

# Attesting the claim as GRANT
data = {
    "action":"GRANT_CLAIM",
}
print(response.json())
attestingClaimId = None
for claim in response.json():
#     print(claim)
    print(claim["status"])
    if claim["status"] == "OPEN":
        attestingClaimId = claim["id"]
        print(claim["id"])
        attestResp = requests.request("POST", "%sTeacher/claims/%s/attest" %(baseUrl, claim["id"]), headers=headers, json=data)
        assert response.status_code == 200
        print(attestResp)
        print(attestResp.json())
        break
assert attestingClaimId != None

In [ ]:

import time
time.sleep(1)
# verify attestation status
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%studentToken,
}

response = requests.request("GET", baseUrl + "Student", headers=headers)


for affiliation in response.json()[0]["studentInstituteAttest"]:
    print(affiliation)
    if affiliation["_osClaimId"] == attestingClaimId:
        assert affiliation["_osState"] == "PUBLISHED"

# Update attested details

In [ ]:
instituteUserId = "admin39@mailinator.com"
instituteRequestBody = {
    "instituteName": "te",
    "email": instituteUserId,
    "contactNumber": "!23"
}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Institute/invite", headers=headers, json=instituteRequestBody)

print(response.text)
assert response.status_code == 200

In [ ]:
token = getToken(instituteUserId)
print(token)

In [47]:
instituteUserId = "admin39@mailinator.com"
instituteRequestBody = {
    "instituteName": "te",
    "email": instituteUserId,
    "contactNumber": "!23"
}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Institute/invite", headers=headers, json=instituteRequestBody)

print(response.text)
assert response.status_code == 200

{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1666058229935,"params":{"resmsgid":"","msgid":"cdac50dd-a469-443b-97c1-8823a7de939b","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Institute":{"osid":"1-10799403-8916-40af-8468-3de640808180"}}}


In [48]:
token = getToken(instituteUserId)
print(token)

{'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJYcGhHaEZVTTlWSUNWZGp2RVlkOTJtQXVoLVYwOWl2YU1ZOHE4cFR5OVRVIn0.eyJleHAiOjE2NjYwNTg4MzAsImlhdCI6MTY2NjA1ODIzMCwianRpIjoiYWNiYTdiMjYtNzI4ZC00ZTBiLWIxYjQtMThkZWIyNjk1NDVmIiwiaXNzIjoiaHR0cHM6Ly9kZW1vLWVkdWNhdGlvbi1yZWdpc3RyeS54aXYuaW4vYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiJkNDI0YzY2OC1hOWM3LTQzMmItYTYzNS1kMTFhMjdkNDNmZjciLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiJhZDZkMTkzMi02NDQyLTRmMGQtOTZkYi02ZjE3MmY2YTRkYTIiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9kZW1vLWVkdWNhdGlvbi1yZWdpc3RyeS54aXYuaW4iLCJodHRwczovL25kZWFyLnhpdi5pbiIsImh0dHA6Ly9sb2NhbGhvc3Q6NDIwMCIsImh0dHA6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vMjAuMTk4LjY0LjEyOCJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtbmRlYXIiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2